In [1]:
## Configurar el ambiente de desarrollo
import pandas as pd
from urllib.parse import unquote, quote
from datetime import datetime


In [2]:
# Leer las tablas desde la URL
url = 'https://es.m.wikipedia.org/wiki/Primera_Divisi%C3%B3n_de_Espa%C3%B1a_2022-23'
all_tables = pd.read_html(url)

# Seleccionar solo las tablas que van del índice 14 al 52 menos la 19 que son las jornadas de la temporada 22-23
selected_tables = all_tables[14:53]
# Elimina la tabla de la posición 19
selected_tables.pop(19)

# Organizar la data
current_results = pd.DataFrame()
for table in selected_tables:
  # Eliminar el primer nivel del MultiIndex de las columnas
  table.columns = table.columns.droplevel(0)
  # Escoger solo las columnas que nos interesan
  table = table[['Local', 'Resultado', 'Visitante', 'Fecha']]
  current_results = pd.concat([current_results, table], ignore_index=True)

# Separar la columna de resultado en goles local / goles visitante
current_results[['GolesLocal', 'GolesVisitante']] = current_results['Resultado'].str.split('–', expand=True)
current_results.drop('Resultado', axis=1, inplace=True)

In [ ]:
current_results

,Local,Visitante,Fecha,GolesLocal,GolesVisitante
0,C. A. Osasuna,Sevilla F. C.,12 de agosto,2,1
1,R. C. Celta de Vigo,R. C. D. Espanyol,13 de agosto,2,2
2,Real Valladolid C. F.,Villarreal C. F.,13 de agosto,0,3
3,F. C. Barcelona,Rayo Vallecano,13 de agosto,0,0
4,Cádiz C. F.,Real Sociedad,14 de agosto,0,1
...,...,...,...,...,...
375,R. C. Celta de Vigo,F. C. Barcelona,4 de junio,NaN,NaN
376,Real Valladolid C. F.,Getafe C. F.,4 de junio,NaN,NaN
377,Elche C. F.,Cádiz C. F.,4 de junio,NaN,NaN
378,R. C. D. Espanyol,U. D. Almería,4 de junio,NaN,NaN


In [3]:
# Borrar las filas que tienen valores NaN
current_results = current_results.dropna()

# Convertir las columnas de GolesLocal y GolesVisitante a tipo número
current_results = current_results.astype({'GolesLocal': int, 'GolesVisitante': int})

current_results.tail

<bound method NDFrame.tail of                      Local          Visitante         Fecha  GolesLocal  \
0            C. A. Osasuna      Sevilla F. C.  12 de agosto           2   
1      R. C. Celta de Vigo  R. C. D. Espanyol  13 de agosto           2   
2    Real Valladolid C. F.   Villarreal C. F.  13 de agosto           0   
3          F. C. Barcelona     Rayo Vallecano  13 de agosto           0   
4              Cádiz C. F.      Real Sociedad  14 de agosto           0   
..                     ...                ...           ...         ...   
255             Real Betis  R. C. D. Mallorca   19 de marzo           1   
256          C. A. Osasuna   Villarreal C. F.   19 de marzo           0   
257          Real Sociedad        Elche C. F.   19 de marzo           2   
258           Getafe C. F.      Sevilla F. C.   19 de marzo           2   
259        F. C. Barcelona  Real Madrid C. F.   19 de marzo           2   

     GolesVisitante  
0                 1  
1                 2  
2  

In [4]:
# Crear función para convertir la fecha
def date_conversion(date_str):
    month_dict = {
        'enero': 1,
        'febrero': 2,
        'marzo': 3,
        'abril': 4,
        'mayo': 5,
        'junio': 6,
        'julio': 7,
        'agosto': 8,
        'septiembre': 9,
        'octubre': 10,
        'noviembre': 11,
        'diciembre': 12
    }
    date_list = date_str.split()
    day = date_list[0]
    month_str = date_list[2]
    month = month_dict[month_str]
    if month <= 7:
        year = 2023
    else:
        year = 2022
    fecha = datetime(year, month, int(day))
    return fecha.strftime('%d/%m/%Y')

# Aplicar la función a la columna Fecha
current_results['Fecha'] = current_results['Fecha'].apply(date_conversion)

current_results['Fecha'] = pd.to_datetime(current_results['Fecha'], format='%d/%m/%Y')
current_results

,Local,Visitante,Fecha,GolesLocal,GolesVisitante
0,C. A. Osasuna,Sevilla F. C.,2022-08-12,2,1
1,R. C. Celta de Vigo,R. C. D. Espanyol,2022-08-13,2,2
2,Real Valladolid C. F.,Villarreal C. F.,2022-08-13,0,3
3,F. C. Barcelona,Rayo Vallecano,2022-08-13,0,0
4,Cádiz C. F.,Real Sociedad,2022-08-14,0,1
...,...,...,...,...,...
255,Real Betis,R. C. D. Mallorca,2023-03-19,1,0
256,C. A. Osasuna,Villarreal C. F.,2023-03-19,0,3
257,Real Sociedad,Elche C. F.,2023-03-19,2,0
258,Getafe C. F.,Sevilla F. C.,2023-03-19,2,0


In [5]:
current_results.dtypes


Local                     object
Visitante                 object
Fecha             datetime64[ns]
GolesLocal                 int64
GolesVisitante             int64
dtype: object

In [6]:
# Extraer el nombre limpio de los equipos que juegan la temporada 2022-23
wiki = 'https://en.wikipedia.org/wiki/'
years = '2022-23'
url = f'{wiki}{years}_La_Liga'

season_2223 = pd.read_html(url)[2]
teams_2223 = list(season_2223['Team'])

# Extraer el nombre a sustituir de los equipos del dataframe
teams_2223_to_change = current_results[['Local', 'Visitante']].values.flatten()
teams_2223_to_change = list(pd.unique(teams_2223_to_change))

# Arreglo con los nuevos nombres de los equipos
renaming_dict = {'C. A. Osasuna': 'Osasuna', 
                 'Sevilla F. C.': 'Sevilla',
                 'R. C. Celta de Vigo': 'Celta Vigo',
                 'R. C. D. Espanyol': 'Espanyol',
                 'Real Valladolid C. F.': 'Valladolid',
                 'Villarreal C. F.': 'Villarreal',
                 'F. C. Barcelona': 'Barcelona',
                 'Rayo Vallecano': 'Rayo Vallecano',
                 'Cádiz C. F.': 'Cádiz',
                 'Real Sociedad': 'Real Sociedad',
                 'Valencia C. F.': 'Valencia',
                 'Girona F. C.': 'Girona',
                 'U. D. Almería': 'Almería',
                 'Real Madrid C. F.': 'Real Madrid',
                 'Athletic Club': 'Athletic Bilbao',
                 'R. C. D. Mallorca': 'Mallorca',
                 'Getafe C. F.': 'Getafe',
                 'Atlético de Madrid': 'Atlético Madrid',
                 'Real Betis': 'Real Betis',
                 'Elche C. F.': 'Elche',
                 'Valencia F. C.': 'Valencia'}

# Sustituir los nombres en el dataframe
current_results.replace(renaming_dict, inplace=True)

In [7]:
# Arreglo con los nuevos nombres de los equipos
renaming_dict = {'C. A. Osasuna': 'Osasuna', 
                 'Sevilla F. C.': 'Sevilla',
                 'R. C. Celta de Vigo': 'Celta Vigo',
                 'R. C. D. Espanyol': 'Espanyol',
                 'Real Valladolid C. F.': 'Valladolid',
                 'Villarreal C. F.': 'Villarreal',
                 'F. C. Barcelona': 'Barcelona',
                 'Rayo Vallecano': 'Rayo Vallecano',
                 'Cádiz C. F.': 'Cádiz',
                 'Real Sociedad': 'Real Sociedad',
                 'Valencia C. F.': 'Valencia',
                 'Girona F. C.': 'Girona',
                 'U. D. Almería': 'Almería',
                 'Real Madrid C. F.': 'Real Madrid',
                 'Athletic Club': 'Athletic Bilbao',
                 'R. C. D. Mallorca': 'Mallorca',
                 'Getafe C. F.': 'Getafe',
                 'Atlético de Madrid': 'Atlético Madrid',
                 'Real Betis': 'Real Betis',
                 'Elche C. F.': 'Elche',
                 'Valencia F. C.': 'Valencia'}

current_results.replace(renaming_dict, inplace=True)


In [8]:
current_results

,Local,Visitante,Fecha,GolesLocal,GolesVisitante
0,Osasuna,Sevilla,2022-08-12,2,1
1,Celta Vigo,Espanyol,2022-08-13,2,2
2,Valladolid,Villarreal,2022-08-13,0,3
3,Barcelona,Rayo Vallecano,2022-08-13,0,0
4,Cádiz,Real Sociedad,2022-08-14,0,1
...,...,...,...,...,...
255,Real Betis,Mallorca,2023-03-19,1,0
256,Osasuna,Villarreal,2023-03-19,0,3
257,Real Sociedad,Elche,2023-03-19,2,0
258,Getafe,Sevilla,2023-03-19,2,0


In [9]:
current_results = current_results.sort_values(by='Fecha', ascending=False)
current_results

,Local,Visitante,Fecha,GolesLocal,GolesVisitante
259,Barcelona,Real Madrid,2023-03-19,2,1
258,Getafe,Sevilla,2023-03-19,2,0
257,Real Sociedad,Elche,2023-03-19,2,0
256,Osasuna,Villarreal,2023-03-19,0,3
255,Real Betis,Mallorca,2023-03-19,1,0
...,...,...,...,...,...
6,Almería,Real Madrid,2022-08-14,1,2
2,Valladolid,Villarreal,2022-08-13,0,3
1,Celta Vigo,Espanyol,2022-08-13,2,2
3,Barcelona,Rayo Vallecano,2022-08-13,0,0


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
pwd

'/content'

In [12]:
cd ./drive/MyDrive/'Colab Notebooks'/laliga-nb

/content/drive/.shortcut-targets-by-id/1Wx_iFi2WCSDzrJxFdNMiae9Ks9HPSpMf/laliga-nb


In [13]:
#Exportar a un archivo .csv
current_results.to_csv('current_results.csv',index=False)

In [14]:
# Crear una función para devolver los últimos N partidos de un equipo en particular

def last_matches_team(df, team, matches_num):
    # Filtrar los partidos del equipo especificado
    team_matches = df[(df['Local'] == team) | (df['Visitante'] == team)]
    
    # Ordenar los partidos por fecha descendente
    team_matches = team_matches.sort_values(by='Fecha', ascending=False)
    
    # Seleccionar los últimos matches_num partidos
    last_matches = team_matches.head(matches_num)
    
    # Devolver el DataFrame con los últimos partidos del equipo
    return last_matches


ultimos_partidos_barcelona =last_matches_team(current_results, 'Almería', 5)
print(ultimos_partidos_barcelona)


       Local   Visitante      Fecha  GolesLocal  GolesVisitante
251  Almería       Cádiz 2023-03-18           1               1
246  Sevilla     Almería 2023-03-12           2               1
232  Almería  Villarreal 2023-03-04           0               2
227  Almería   Barcelona 2023-02-26           1               0
210   Girona     Almería 2023-02-17           6               2
